# Implement the forward propagation for a two hidden layer network for m-samples, n-features as we discussed in class. Initialize the weights randomly. Use the data from the previous labs like logistic regression. You can choose the number of neurons in the hidden layer and use sigmoid activation function.Report the evaluation metrics for the network.  Also use other non-linear activation functions like ReLU and Tanh. Report the loss using both MSE and Cross Entropy.

In [1]:
import pandas as pd
df = pd.read_csv("Logistic_regression_ls.csv")
df

,x1,x2,label
0,7.395,7.638,1
1,4.987,6.485,1
2,5.358,6.499,1
3,2.036,2.380,0
4,5.956,7.378,1
...,...,...,...
495,0.304,1.608,0
496,6.140,4.261,1
497,6.579,6.231,1
498,2.555,0.446,0


In [81]:
num_samples = df[['x1','x2']].values
print(num_samples.T.shape)
y = df[['label']].values
print(y.shape)
num_samples.shape[1]


(2, 500)
(500, 1)


2

In [82]:
import numpy as np

def sigmoid(x):
    return 1/(1+np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def tanh(x):
    return np.tanh(x)

def activation(x,activ):
    if activ == 'sigmoid':
        return 1/(1+np.exp(-x)) 
    if activ == 'relu':
        return np.maximum(0, x)
    if activ == 'tanh':
        return np.tanh(x) 
    
def mean_squard_loss(y_true,y_pred):
    loss = np.mean((y_true - y_pred)**2)
    return loss
    
def cross_entropy_loss(y_true,y_pred):
    eps = 1e-5
    y_pred = np.clip(y_pred,eps,1-eps)
    yp_log1 = np.log(y_pred)
    yp_log2 = np.log(1-y_pred)
    loss = -np.mean(y_true*yp_log1) - np.mean((1 - y_true)*yp_log2)
    return loss

# Input_size is the number of features 
input_size = num_samples.shape[1]
num_hidden_layers = 2
nodes_in_hidden_layers = [10,4]
output_size = 1

w1 = np.random.rand(input_size, nodes_in_hidden_layers[0])  # shape of w1 (2,10)
b1 = np.random.rand(nodes_in_hidden_layers[0],1) # shape of b1 (10,1)

w2 = np.random.rand(nodes_in_hidden_layers[0],nodes_in_hidden_layers[1])  # shape of w2 (10,4)
b2 = np.random.rand(nodes_in_hidden_layers[1],1)  # shape of w1 (4,1)

w3 = np.random.rand(nodes_in_hidden_layers[1],output_size) # shape of w3 (1,1)
b3 = np.random.rand(output_size,1)   # shape of w1 (1,1)

def forward_prop(x,w1,w2,w3,b1,b2,b3, activ):
                                # shape of x (2,500) and number of nodes in 1st and 2nd hidden layers are 10 and 4
    a1 = np.dot(w1.T,x) + b1    # shape of a1 (10,500)
    h1 = activation(a1,activ)   # shape of h1 (10,500)
    
    a2 = np.dot(w2.T,h1) + b2   # shape of a2 (4,500)
    h2 = activation(a2,activ)   # shape of h2 (4,500)
    
    a3 = np.dot(w3.T, h2) + b3  # shape of a3 (1,500)
    y_pred = sigmoid(a3)  # shape of a1 (1,500)
    
    return y_pred.T      # output shape (500,1)

x = num_samples.T  # shape of x (2,500)
y_true = y     # shape of y_true (500,1)
y_pred1 = forward_prop(x,w1,w2,w3,b1,b2,b3, 'sigmoid')   # shape of y_pred (500,1)
y_pred2 = forward_prop(x,w1,w2,w3,b1,b2,b3, 'tanh')
y_pred3 = forward_prop(x,w1,w2,w3,b1,b2,b3, 'relu')

print(f"Mean squared loss for activation sigmoid: {mean_squard_loss(y_true,y_pred1):.4f}" )
print(f"Mean squared loss for activation tanh: {mean_squard_loss(y_true,y_pred2):.4f}" )
print(f"Mean squared loss for activation relu: {mean_squard_loss(y_true,y_pred3):.4f}" )

print(f"cross entropy loss for activation sigmoid: {cross_entropy_loss(y_true,y_pred1):.4f}" )
print(f"cross entropy loss for activation tanh: {cross_entropy_loss(y_true,y_pred2):.4f}" )
print(f"cross entropy loss for activation relu: {cross_entropy_loss(y_true,y_pred3):.4f}" )



Mean squared loss for activation sigmoid: 0.4262
Mean squared loss for activation tanh: 0.4268
Mean squared loss for activation relu: 0.5000
cross entropy loss for activation sigmoid: 1.3034
cross entropy loss for activation tanh: 1.3075
cross entropy loss for activation relu: 5.7191


In [83]:
# Define evaluation metrics
def evaluate_metrics(y_true, y_pred):
    # Calculate True Positives, False Positives, True Negatives, False Negatives
    TP = np.sum((y_true == 1) & (y_pred == 1))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FN = np.sum((y_true == 1) & (y_pred == 0))

    # Calculate Precision, Recall, F1-Score, Accuracy
    P = TP / (TP + FP) if (TP + FP) != 0 else 0
    R = TP / (TP + FN) if (TP + FN) != 0 else 0
    f1 = 2 * (P * R) / (P + R) if (P + R) != 0 else 0
    acc = (TP + TN) / len(y_true)
    return P, R, f1, acc

In [84]:
# converting the output probabilities to labels 0 or 1 for probability greater than 0.5
y_pred1_labels = np.where(y_pred1 > 0.5, 1, 0)
y_pred2_labels = np.where(y_pred2 > 0.5, 1, 0)
y_pred3_labels = np.where(y_pred3 > 0.5, 1, 0)

In [85]:
precision1, recall1, f1_score1, accuracy1 = evaluate_metrics(y_true, y_pred1_labels)
print("Evaluation matrix for activation sigmoid")
print("Precision: ",precision1)
print("Recall: ",recall1)
print("F1_score: ",f1_score1)
print(f"Accuracy: {precision1*100}%")
print()
precision2, recall2, f1_score2, accuracy2 = evaluate_metrics(y_true, y_pred2_labels)
print("Evaluation matrix for activation Tanh")
print("Precision: ",precision2)
print("Recall: ",recall2)
print("F1_score: ",f1_score2)
print(f"Accuracy: {precision2*100}%")
print()
precision3, recall3, f1_score3, accuracy3 = evaluate_metrics(y_true, y_pred3_labels)
print("Evaluation matrix for activation relu")
print("Precision: ",precision3)
print("Recall: ",recall3)
print("F1_score: ",f1_score3)
print(f"Accuracy: {precision3*100}%")

Evaluation matrix for activation sigmoid
Precision:  0.5
Recall:  1.0
F1_score:  0.6666666666666666
Accuracy: 50.0%

Evaluation matrix for activation Tanh
Precision:  0.5
Recall:  1.0
F1_score:  0.6666666666666666
Accuracy: 50.0%

Evaluation matrix for activation relu
Precision:  0.5
Recall:  1.0
F1_score:  0.6666666666666666
Accuracy: 50.0%
